# Módulo 2: Scraping con Selenium
## LATAM Airlines
<a href="https://www.latam.com/es_ar/"><img src="https://i.pinimg.com/originals/dd/52/74/dd5274702d1382d696caeb6e0f6980c5.png"  width="420"></img></a>
<br>

Vamos a scrapear el sitio de Latam para averiguar datos de vuelos en funcion el origen y destino, fecha y cabina. La información que esperamos obtener de cada vuelo es:
- Precio(s) disponibles
- Horas de salida y llegada (duración)
- Información de las escalas

**¡Empecemos!**
Utilicemos lo aprendido hasta ahora para lograr el objetivo propuesto

## Selenium

Selenium es una herramienta que nos permitirá controlar un navegador y podremos utilizar las funcionalidades del motor de JavaScript para cargar el contenido que no viene en el HTML de la página. Para esto necesitamos el módulo `webdriver`.

In [1]:
from selenium import webdriver
#importampos libreria para cargar el driver automaticamente
from webdriver_manager.firefox import GeckoDriverManager

url='https://www.latamairlines.com/py/es/ofertas-vuelos?origin=ASU&outbound=2023-08-01T12%3A00%3A00.000Z&destination=BCN&inbound=null&adt=1&chd=0&inf=0&trip=OW&cabin=Economy&redemption=false&sort=RECOMMENDED'

Necesitamos controladores web para diferentes navegadores web.<br>
Paso 1: instanciar un **driver** del navegador

In [2]:
options = webdriver.FirefoxOptions()
# Podemos agregarle opciones al driver para utilizar los distintos modos del navegador
options.add_argument('-private')
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options)

[WDM] - Downloading: 19.0kB [00:00, 6.52MB/s]                   
[WDM] - Downloading: 100%|██████████| 1.58M/1.58M [00:01<00:00, 1.08MB/s]
C:\Users\mario\AppData\Local\Temp\ipykernel_14396\1250048064.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options)


Paso 2: hacer que el navegador cargue la página web.

In [3]:
driver.get(url)

paso 3:Extraer informacion de la pagina<br>
Carguemos la página y analicemos dónde se encuentra la información<br>
Vemos que el bloque de vuelos se encuentra en una `ul` y que cada vuelo es un item de la lista, `li`. <br>

In [4]:
#Usaremos el Xpath para obtener la lista de vuelos
vuelos = driver.find_elements('xpath','//ol/li')
print (vuelos)

[<selenium.webdriver.remote.webelement.WebElement (session="aedd09d9-9eeb-4b9b-baa7-a9f84535d58f", element="6683f637-efbe-4939-b40d-7df0ebb61d20")>, <selenium.webdriver.remote.webelement.WebElement (session="aedd09d9-9eeb-4b9b-baa7-a9f84535d58f", element="986234df-1a00-4d7f-8a8e-be1ab01b540d")>, <selenium.webdriver.remote.webelement.WebElement (session="aedd09d9-9eeb-4b9b-baa7-a9f84535d58f", element="1691c09f-e1cb-4b39-a556-3a652ce57a46")>, <selenium.webdriver.remote.webelement.WebElement (session="aedd09d9-9eeb-4b9b-baa7-a9f84535d58f", element="44e9f267-e022-4d36-831c-b12bcdc53ba9")>, <selenium.webdriver.remote.webelement.WebElement (session="aedd09d9-9eeb-4b9b-baa7-a9f84535d58f", element="0f63cd7e-15de-4e84-b115-287b83a63af1")>, <selenium.webdriver.remote.webelement.WebElement (session="aedd09d9-9eeb-4b9b-baa7-a9f84535d58f", element="547241cf-6246-4204-8e55-7e093c5a0b6b")>, <selenium.webdriver.remote.webelement.WebElement (session="aedd09d9-9eeb-4b9b-baa7-a9f84535d58f", element="901a

Obtengamos la informacion de la hora de salida, llegada y duracion del vuelo

Notar que el xpath comienza con ".", lo que indica que sólo debe buscar en los hijos de ese elemento. Si no ponemos el ".", busca en todo el árbol.<br>
Elementos importantes de xpath:
- "//" busca en todos los hijos del elemento. "/" busca sólo en hijos directos
- "." indica que la búsqueda debe empezar en ese elemento y no en el origen del árbol
- Los atributos de los tags se buscan entre [] y con @

In [10]:
#seleccionamos el primer vuelo
vuelo_1=vuelos[0]
#hora de salida
hora_salida=vuelo_1.find_element('xpath','//div[@class="sc-klSiHT hjzFuR flight-information"]/span[1]').text
print (hora_salida)

6:55 a. m.


In [11]:
#hora de llegada
hora_llegada=vuelo_1.find_element('xpath','.//div[3]/span[1]').text.replace('\n+1','')
print (hora_llegada)

5:20 p. m.


In [12]:
# Duracion del vuelo
duracion_vuelo= vuelo_1.find_element('xpath','.//div[2]/span[2]').text
print (duracion_vuelo)

28 h 25 min


veremos cómo obtener la información de las escalas de cada vuelo. <br>
Para desplegar esa información de las escalas, debemos clickear en link para que se habilite el modal que contiene la informacion. Seleccionémoslo:

In [14]:
link_escalas = vuelo_1.find_element('xpath','//div[@class="sc-iKiVwC fbWfQZ"]//a')
print (link_escalas)

<selenium.webdriver.remote.webelement.WebElement (session="aedd09d9-9eeb-4b9b-baa7-a9f84535d58f", element="afe06b17-fdd4-4b3e-bd85-fcde17b97866")>


Una vez ubicado el elemento podemos clickear sobre él

In [15]:
link_escalas.click()

Y vemos cómo se despliega la información que estamos buscando enn el driver del navegador. **Notar que cambió el html de la página al hacer click sobre ese botón**

ahora debemos seleccionar los segmentos que contiene el vuelo para calcular la cantidad de escalas, actualmente vemos que las paradas estan contenidas en elementos sections, asi que debemos contabilizar las paradas

In [19]:
paradas= link_escalas.find_elements('xpath','//section[@class="sc-fGSyRc fCuylQ"]')
print(paradas)

[<selenium.webdriver.remote.webelement.WebElement (session="aedd09d9-9eeb-4b9b-baa7-a9f84535d58f", element="ca8ab4e1-e1a3-40cd-b886-8539de157a5c")>, <selenium.webdriver.remote.webelement.WebElement (session="aedd09d9-9eeb-4b9b-baa7-a9f84535d58f", element="85afdffb-9260-4c34-86d7-8108430a9435")>, <selenium.webdriver.remote.webelement.WebElement (session="aedd09d9-9eeb-4b9b-baa7-a9f84535d58f", element="9bcad1de-bc1f-422d-9487-dfcdbb293104")>]


para calcular las escalas, debemos restar -1 la cantidad total de paradas

In [20]:
escalas=len(paradas)-1
print(escalas)

2


Hasta aquí llegamos en esta seccion, en la que vimos cómo utilizar Selenium para intearctuar con elementos de la pàgina web para obtener información que estaba "oculta"

Paso 4: cerrar el navegador

In [21]:
driver.close()